In [34]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import load_model
from keras import layers
from keras.models import Model, Input
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix


In [2]:
df = pd.read_csv('./fer2013/fer2013.csv', dtype={'emotion':np.int32, 'pixels':str, 'Usage':str})

In [3]:
df['pixels'] = df['pixels'].apply(lambda x: np.fromstring(x,sep=' '))

In [4]:
train = df.loc[df['Usage'] == 'Training']
validation = df.loc[df['Usage'] == 'PublicTest']
test = df.loc[df['Usage'] == 'PrivateTest']

y_train = pd.get_dummies(train['emotion'])
y_train.columns = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

y_val = pd.get_dummies(validation['emotion'])
y_val.columns = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

y_test = pd.get_dummies(test['emotion'])
y_test.columns = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

In [5]:
x_train = np.vstack(train['pixels'].values)
x_validation = np.vstack(validation['pixels'].values)
x_test = np.vstack(test['pixels'].values)

In [6]:
X_train = np.stack((np.reshape(x_train,(-1, 48, 48, 1)),)*3, axis=-2).squeeze()
X_val = np.stack((np.reshape(x_validation,(-1, 48, 48, 1)),)*3, axis=-2).squeeze()
X_test =  np.stack((np.reshape(x_test,(-1, 48, 48, 1)),)*3, axis=-2).squeeze()

In [7]:
with tf.device('/cpu:0'):
    tf_x_train = tf.placeholder(tf.float32, shape=(len(X_train), 48, 48, 3))
    tf_x_validation = tf.placeholder(tf.float32, shape=(len(X_val), 48, 48, 3))
    tf_x_test = tf.placeholder(tf.float32, shape=(len(X_test), 48, 48, 3))

    tf_x_train_resized = tf.image.resize_images(tf_x_train,  size=[75,75])
    tf_x_validation_resized = tf.image.resize_images(tf_x_validation,  size=[75,75])
    tf_x_test_resized = tf.image.resize_images(tf_x_test,  size=[75,75])

In [8]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    X_train,X_val,X_test = sess.run([tf_x_train_resized,tf_x_validation_resized,tf_x_test_resized], feed_dict={tf_x_train: X_train,
                                                   tf_x_validation: X_val,
                                                   tf_x_test: X_test
                                                  })

In [10]:
models_path =['./vgg16_fine_tuning.h5','./resnetv2_fine_tunning.h5','./xception_fine_tunning.h5']

In [11]:
models=[]
for model_path in models_path:

    modelTemp=load_model(model_path) # load model
    modelTemp.name=model_path # change name to be unique
    models.append(modelTemp)

In [24]:
def ensembleModels(models, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 

    # averaging outputs
    yAvg=layers.average(yModels) 

    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg,    name='ensemble')  
   
    return modelEns

def accuracy_normalized(y_true,y_pred):
    mc = confusion_matrix(y_true,y_pred)
    mc_margin = np.sum(mc,axis = 1)

    n = len(mc)

    tp = []
    for i in range(n):
        tp.append(mc[i][i]/mc_margin[i])

    accuracy_normalized = np.sum(tp)/n 

    return accuracy_normalized

def acuracia(y_true,y_pred):
    mc = confusion_matrix(y_true,y_pred)
    mc_tot = np.sum(mc)
    n = len(mc)

    tp = []
    for i in range(n):
        tp.append(mc[i][i])

    accuracy = np.sum(tp)/mc_tot 

    return accuracy

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de confusão',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=16)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=12)
    plt.yticks(tick_marks, classes, fontsize=12)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Observado', fontsize=14)
    plt.xlabel('Predito', fontsize=14)
    plt.tight_layout()

In [13]:
model_input = Input(shape=models[0].input_shape[1:])
modelEns = ensembleModels(models, model_input)
modelEns.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
./vgg16_fine_tuning.h5 (Model)  (None, 7)            14729031    input_1[0][0]                    
__________________________________________________________________________________________________
./resnetv2_fine_tunning.h5 (Mod (None, 7)            54347495    input_1[0][0]                    
__________________________________________________________________________________________________
./xception_fine_tunning.h5 (Mod (None, 7)            20990511    input_1[0][0]                    
__________________________________________________________________________________________________
average_1 

In [79]:
def model_output(model,X,y):
    pred = model.predict(X/255.)
    pred_argmax = np.argmax(pred,axis=1)
    pred_onehot = np.zeros_like(pred)
    pred_onehot[np.arange(len(pred)), pred.argmax(1)] = 1
    y_argmax = np.argmax(np.array(y),axis=1)
    return acuracia(y_argmax,pred_argmax), accuracy_normalized(y_argmax,pred_argmax)

In [81]:
model_output(models[0], X_val, y_val)

(0.6770688213987183, 0.6437437282351909)

In [82]:
model_output(models[1], X_val, y_val)

(0.47227640011145167, 0.39832939367006454)

In [83]:
model_output(models[2], X_val, y_val)

(0.6614655893006408, 0.6203530683133517)

In [84]:
model_output(modelEns, X_val, y_val)

(0.6837559208693229, 0.6359777465925843)

In [85]:
model_output(modelEns, X_test, y_test)

(0.6904430203399275, 0.6541872557857765)

In [ ]:
#modelEns.save(path2ensModel)

In [ ]:
#modelEns=load_model(path2ensModel)
#modelEns.summary()
#y=modelEns.predict(x)